# Demonstrate How to Combine Multiple Tanks into a Network

In [1]:
from cas_models.discrete_time.models import StateSpaceModelDT
from cas_models.transformations import connect_systems
from feed_conc_ctrl import MixingTankModelDT

In [2]:
D = 5  # tank diameter [m]
n_tanks = 4
tank_names = [f"tank_{i+1}" for i in range(n_tanks)]

systems = [MixingTankModelDT(D=D, name=name) for name in tank_names]

def print_sys_dimensions(sys):
    print(sys.name, f"({sys.ny}x{sys.nu})")
    for attr_name in ["input_names", "state_names", "output_names"]:
        print(f"{attr_name:>15s}: {getattr(sys, attr_name)}")

for sys in systems:
    print_sys_dimensions(sys)
    

tank_1 (3x3)
    input_names: ['v_dot_in', 'conc_in', 'v_dot_out']
    state_names: ['L', 'm']
   output_names: ['L', 'm', 'conc_out']
tank_2 (3x3)
    input_names: ['v_dot_in', 'conc_in', 'v_dot_out']
    state_names: ['L', 'm']
   output_names: ['L', 'm', 'conc_out']
tank_3 (3x3)
    input_names: ['v_dot_in', 'conc_in', 'v_dot_out']
    state_names: ['L', 'm']
   output_names: ['L', 'm', 'conc_out']
tank_4 (3x3)
    input_names: ['v_dot_in', 'conc_in', 'v_dot_out']
    state_names: ['L', 'm']
   output_names: ['L', 'm', 'conc_out']


In [3]:
help(connect_systems)

Help on function connect_systems in module cas_models.transformations:

connect_systems(
    systems,
    connections,
    model_class,
    input_names=None,
    output_names=None,
    keys=None,
    verbose_names=False,
    prefix='sys',
    name=None,
    sep='_'
)
    Connect multiple nonlinear systems with arbitrary connections.

    This function combines multiple state-space models (continuous-time or
    discrete-time) into a single connected system by:
    1. First arranging all systems in parallel
    2. Creating internal connections between outputs and inputs
    3. Exposing only specified external inputs and outputs

    Args:
        systems (list): List of state-space model objects to connect.
        connections (list or dict): Connection specification:
            - List format: [(source_name, input_name), ...] for simple
                one-to-one connections where source_name is an output name
                or external input name.
            - Dict format: {input_na

In [4]:
connections = {
    'tank_2_conc_in': 'tank_1_conc_out',
    'tank_3_conc_in': 'tank_1_conc_out',
    'tank_4_conc_in': ['tank_2_conc_out', 'tank_3_conc_out'],
    'tank_1_v_dot_out': ['tank_2_v_dot_in', 'tank_3_v_dot_in'],
    'tank_4_v_dot_in': ['tank_2_v_dot_out', 'tank_3_v_dot_out'],
}
model_class = StateSpaceModelDT
feed_tanks_system = connect_systems(systems, connections, model_class, name="tank_system_121") 
print_sys_dimensions(feed_tanks_system)

tank_system_121 (12x7)
    input_names: ['tank_1_v_dot_in', 'tank_1_conc_in', 'tank_2_v_dot_in', 'tank_2_v_dot_out', 'tank_3_v_dot_in', 'tank_3_v_dot_out', 'tank_4_v_dot_out']
    state_names: ['tank_1_L', 'tank_1_m', 'tank_2_L', 'tank_2_m', 'tank_3_L', 'tank_3_m', 'tank_4_L', 'tank_4_m']
   output_names: ['tank_1_L', 'tank_1_m', 'tank_1_conc_out', 'tank_2_L', 'tank_2_m', 'tank_2_conc_out', 'tank_3_L', 'tank_3_m', 'tank_3_conc_out', 'tank_4_L', 'tank_4_m', 'tank_4_conc_out']
